Install and import neccessary libraries. This implementation uses Pandas (and GeoPandas) as well as geopy for geocoding.

In [ ]:
!pip install pandas
!pip install folium matplotlib mapclassify geopandas
!pip install geopy

import pandas as pd
import geopandas as gpd
import geopy as gp

import os
import time

In [ ]:
# Setup geocoding provider

from geopy.geocoders import GoogleV3
API_KEY = "<insert api key>"

geolocator = GoogleV3(api_key=API_KEY)

Configure variables used to define input data format. Change these if changing the format of the election data.

In [ ]:
COUNTRY = "Hrvatska"

ADDRESS_COLUMN = "Adresa BM"
CITY_COLUMN = "Grad/općina/država"

POINT_COLUMN = "geometry"

Firstly, load and concatenate all election voting spot locations and convert their addresses to coordinates.

In [ ]:
dataset = gpd.GeoDataFrame()

for (path, dirnames, filenames) in os.walk("data"):
  for filename in filenames:
    frame = pd.read_excel(os.sep.join([path, filename]))

    dataset = pd.concat([dataset, frame])

print(dataset)

In [ ]:
relevant_dataset = dataset.loc[:, [ADDRESS_COLUMN, CITY_COLUMN]] # Keep only the index and two location columns

def make_complete_address_column(row):
  return str(row[ADDRESS_COLUMN] + ", " + row[CITY_COLUMN] + ", " + COUNTRY)

relevant_dataset[ADDRESS_COLUMN] = relevant_dataset.apply(make_complete_address_column, axis=1)
relevant_dataset = relevant_dataset.drop(CITY_COLUMN, axis=1) # CITY_COLUMN data is now a part of ADDRESS_COLUMN

print(relevant_dataset)

The remaining column should be enough to geocode the address into the coordinates of the voting place

In [ ]:
point_column_data = []

#relevant_dataset = relevant_dataset.drop(POINT_COLUMN, axis=1)

for index, row in relevant_dataset.iterrows():
  result = geolocator.geocode(row[ADDRESS_COLUMN], exactly_one=True)
  point_column_data.append(result.point)

  print(result)

point_frame = gpd.GeoDataFrame({ POINT_COLUMN: point_column_data })
relevant_dataset = relevant_dataset.join(point_frame)

print(relevant_dataset)

In [ ]:
out_format = pd.DataFrame(relevant_dataset);
out_format["longitude"] = out_format[POINT_COLUMN].apply(lambda x: x.longitude)
out_format["latitude"] = out_format[POINT_COLUMN].apply(lambda x: x.latitude)

out_format = out_format.drop(POINT_COLUMN, axis=1)

print(out_format)

out_format.to_csv("out_data.csv")